In [27]:

# A neural network is a network or circuit of neurons, or in a modern sense, an artificial neural network, composed of artificial neurons or nodes.  Thus a neural network is either a biological neural network, made up of real biological neurons, or an artificial neural network, for solving artificial intelligence (AI) problems.

# These artificial networks may be used for predictive modeling, adaptive control and applications where they can be trained via a dataset.  Self-learning resulting from experience can occur within networks, which can derive conclusions from a complex and seemingly unrelated set of information.

from pandas_datareader import data as wb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
from sklearn.preprocessing import MinMaxScaler


start = '2019-06-30'
end = '2020-06-30'

tickers = ['SBUX']

thelen = len(tickers)

price_data = []
for ticker in tickers:
    prices = wb.DataReader(ticker, start = start, end = end, data_source='yahoo')[['Open','Adj Close']]
    price_data.append(prices.assign(ticker=ticker)[['ticker', 'Open', 'Adj Close']])

#names = np.reshape(price_data, (len(price_data), 1))

df = pd.concat(price_data)
df.reset_index(inplace=True)

for col in df.columns: 
    print(col) 
    
#used for setting the output figure size
rcParams['figure.figsize'] = 20,10
#to normalize the given input data
scaler = MinMaxScaler(feature_range=(0, 1))
#to read input data set (place the file name inside  ' ') as shown below
df.head()

df['Date'] = pd.to_datetime(df.Date,format='%Y-%m-%d')
# df.index = names['Date']
#3plt.figure(figsize=(16,8))
# plt.plot(df['Adj Close'], label='Closing Price')


ntrain = 80
df_train = df.head(int(len(df)*(ntrain/100)))
ntest = -80
df_test = df.tail(int(len(df)*(ntest/100)))


#importing the packages 
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

#dataframe creation
seriesdata = df.sort_index(ascending=True, axis=0)
new_seriesdata = pd.DataFrame(index=range(0,len(df)),columns=['Date','Adj Close'])
length_of_data=len(seriesdata)
for i in range(0,length_of_data):
    new_seriesdata['Date'][i] = seriesdata['Date'][i]
    new_seriesdata['Adj Close'][i] = seriesdata['Adj Close'][i]
#setting the index again
new_seriesdata.index = new_seriesdata.Date
new_seriesdata.drop('Date', axis=1, inplace=True)
#creating train and test sets this comprises the entire data’s present in the dataset
myseriesdataset = new_seriesdata.values
totrain = myseriesdataset[0:255,:]
tovalid = myseriesdataset[255:,:]
#converting dataset into x_train and y_train
scalerdata = MinMaxScaler(feature_range=(0, 1))
scale_data = scalerdata.fit_transform(myseriesdataset)
x_totrain, y_totrain = [], []
length_of_totrain=len(totrain)
for i in range(60,length_of_totrain):
    x_totrain.append(scale_data[i-60:i,0])
    y_totrain.append(scale_data[i,0])
x_totrain, y_totrain = np.array(x_totrain), np.array(y_totrain)
x_totrain = np.reshape(x_totrain, (x_totrain.shape[0],x_totrain.shape[1],1))


#LSTM neural network
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(x_totrain.shape[1],1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_squared_error', optimizer='adadelta')
lstm_model.fit(x_totrain, y_totrain, epochs=10, batch_size=1, verbose=2)
#predicting next data stock price
myinputs = new_seriesdata[len(new_seriesdata) - (len(tovalid)+1) - 60:].values
myinputs = myinputs.reshape(-1,1)
myinputs  = scalerdata.transform(myinputs)
tostore_test_result = []
for i in range(60,myinputs.shape[0]):
    tostore_test_result.append(myinputs[i-60:i,0])
tostore_test_result = np.array(tostore_test_result)
tostore_test_result = np.reshape(tostore_test_result,(tostore_test_result.shape[0],tostore_test_result.shape[1],1))
myclosing_priceresult = lstm_model.predict(tostore_test_result)
myclosing_priceresult = scalerdata.inverse_transform(myclosing_priceresult)

    
totrain = df_train
tovalid = df_test

#predicting next data stock price
myinputs = new_seriesdata[len(new_seriesdata) - (len(tovalid)+1) - 60:].values

#  Printing the next day’s predicted stock price. 
print(len(tostore_test_result));
print(myclosing_priceresult);



Date
ticker
Open
Adj Close
Epoch 1/10
193/193 - 3s - loss: 0.2868
Epoch 2/10
193/193 - 3s - loss: 0.2671
Epoch 3/10
193/193 - 3s - loss: 0.2465
Epoch 4/10
193/193 - 3s - loss: 0.2253
Epoch 5/10
193/193 - 3s - loss: 0.2040
Epoch 6/10
193/193 - 3s - loss: 0.1827
Epoch 7/10
193/193 - 3s - loss: 0.1617
Epoch 8/10
193/193 - 3s - loss: 0.1413
Epoch 9/10
193/193 - 3s - loss: 0.1217
Epoch 10/10
193/193 - 3s - loss: 0.1033
1
[[67.15351]]


In [ ]:

# In this experiment we took one calendar year of data, fed it into the RNN model, and asked it to predict the next day's closing price.  The time series started on 6/30/2019 and ended on 6/30/2020, so we are trying to predict the closing price for 07/01/2020.  The model said it would be 67.15.  On 07/01/2020, SBUX actually closed at 74.03.  The stock closed around 10% hgher than the model predicted.  Although this sounds great, and we would have hypothetically made an extra 10% on our investment, we would have preferred to see the model make a better prediction, which perhaps, would be off by 1% or 2%, but not 10%.


In [ ]:

# data:
# All data is imported from Yahoo Finance.  Yahoo sources the data from Capital IQ.